# Optimisation Algorithms Part I
> Some most commonly used optimization algorithms for updating network parameters, and their advantages/disadvantages.

- toc: true 
- badges: true
- comments: true

## Introduction
There are millions of parameters in a typical deep neural network, and these parameters are initialized with random values. Now as the training proceeds, as mentioned in [the previous post](https://nitishsadire.github.io/DLblog/jupyter/2020/10/14/DLforCNN.html), we fed the training examples into the network and compute loss using a loss function, and take gradients of loss w.r.t. all the parameters so as to update them. Now with the updated network parameters we again fed the training data, and this process repeats time and again until the loss converges. The part where we update the parameters of the network is the focus of today's post.

To update a parameter means to change it's value so as to minimize the loss we incurred. It is an optimization problem where the objective function is the loss function $L(w)$, $w$ is the parameter we are updating, and ideally, we want to find the global minima of loss function $L$ for parameter w, if such global minima exist, otherwise find local minima. One such method to find a minima. as mentioned in [the previous post](https://nitishsadire.github.io/DLblog/jupyter/2020/10/14/DLforCNN.html), as Gradient Descent (GD) also known as Vanilla Gradient Descent, where we take a small step in the direction of the negative gradient ($-\frac{\partial{L}}{\partial{w}}$). There are numerous issues with GD, like:
1. Too much memory is required to feed all the training examples at once to the model.
1. What if we reached a saddle point (a point which is not local extremum but having gradient 0, please refer [wikipedia](https://en.wikipedia.org/wiki/Saddle_point)), then gradient will be zero there and no updation will happen. 

Because of such issues, it's hardly used in practice. Other optimization algorithms have fundamental ideas the same as gradient descent but they encounter the issues faced by it. 



## Stochastic Gradient Descent (SGD)
It's a computational improvement over gradient descent as rather than feeding the whole training data it feeds data in batches. So if there are n training examples, then the batch size $b$ could be $1\leq b \leq n$. Batch-size is chosen based on memory availability. Mathematically,

$x_{t+1} = x_t - \alpha.\frac{\partial{L}}{\partial{x_t}}$

Some of the issues with SGD are:
- Not whole data is fed at once, so gradients will be noisy. By noisy I mean that because each batch will have a different set of examples, so they try to change the parameters in different directions, therefore there won't be a smooth curve to the minima, as shown below.


![](images/blog3_1.png "Credit:cs231n")

- What if the rate of change of loss is different in different directions. For example, as shown below, the contours of loss function form an ellipse, and function changes faster along the y-axis than the x-axis. Therefore the magnitude of the gradient along the y-axis will be more than the x-axis, and so the net direction of the gradient will align more with the y-axis. Therefore, the negative of the gradient will also align more with the y-axis, and hence the SGD will follow a zigzag path, which will take longer to converge.

![](images/blog3_2.png "Credit:cs231n")

- Issues with saddle points still exists.

## SGD + Momentum
In SGD with Momentum, at any time t+1, the direction is not determined by $x_t$, but all the gradients from the beginning. Mathematically, if $v_0 = 0$ then,

$v_{t+1} = \rho v_t + \frac{\partial{L}}{\partial{x_t}}$

$x_{t+1} = x_t - \alpha.v_{t+1}$

typically, $\rho$=0.9, or 0.99. $v_t$ is known as velocity which is actually the running mean of gradients. Velocity at a point has exponentially decaying weighted sum of previous gradients (older the gradient is less is its weight). Therefore it's less noisy than SGD because gradients of other batches are also added. 

Also in case of different rates of change of loss along with different dimensions, velocity gives better direction, plus the dimension along which there is less rate of change of loss function will have better velocity than in a normal SGD. So convergence will be faster as the path will be less zigzag. The velocity increases for dimension whose gradients point in the same direction and reduces for dimensions whose updates are in different directions. Therefore there will be less oscillation while converging.

It won't get stuck in saddle point because of velocity. Also, it will overshoot local minima but will come back and approach the minima again.


## Nesterov Momentum
One issue with SGD+Momentum is that the gradients could accumulate which leads to a high gradient in a particular direction, which is not good because it could surpass the local minima, also convergence will be slower as it has to move back and forth to reach a local minima. Imagine like a ball is running downhill, then SGD+Momentum could be thought of as ball accumulating very high velocity, so could surpass a local minima and will try to climb an uphill. 
Nesterov Momentum update tries to correct this "blindly following the slope" nature of SGD+Momentum by calculating the slope of the future point, and moving in the -ve of net gradient direction. Mathematically,

$v_{t+1} = \rho v_t + \alpha\nabla_{x_t}L(x_t - \rho v_t)$

$x_{t+1} = x_t - v_{t+1}$

So here, firstly gradient of loss function L w.r.t to $x_t$ is computed at the approximate future point $x_t - \rho v_t$ (it's approximate because actual future point is $x_t - \rho v_t -$ gradient_of_L_w.r.t_x_t), so we have gradient knowledge of the next point. Now we add accumulated velocities to it and make an update on x. 

This anticipatory update prevents us from going too fast and results in better updates, which has significantly increased performance on DL models. Now, we are able to adapt our updates to the slope of our error function and speed up SGD in turn.

SGD+Momentum and Nesterov Momentum updates are shown below in the figure. SGD+M computes gradient first (small blue vector) and then takes a step in the net direction of -ve accumulated gradients (big blue vector). Whereas, Nesterov goes in the direction of previously accumulated gradients (big brown vector), then finds gradient at approximate next point, and takes a step in -ve gradient direction (small red vector), so that the resultant direction (big green vector) is different from SGD+M.

![](images/blog3_3.png)

## Conclusion
We have covered Vanilla GD, SGD, SGD+Momentum, and Nesterov Momentum algorithms. All these optimization algorithms need a fine learning rate $\alpha$. If $\alpha$ too big then it will surpass the local minima or even diverge, if it's too small then convergence will be very slow. In Part II, we'll look at adaptive optimization algorithms, that will manage the learning rate for every parameter by itself, which is a very huge benefit over non-adaptive algorithms we have covered so far.